# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [4]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [5]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'C:\\Users\\acer\\Desktop\\spring1819_assignment1\\assignment1\\cs231n\datasets\\cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()


## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for your own interest.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [6]:
from cs231n.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])


Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [7]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [5e-7,1e-6,2e-6]
regularization_strengths = [5e-4, 1e-3, 2e-3]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
itertimes = 5001
for learningrate in learning_rates:
    for reg in regularization_strengths:
        clf = LinearSVM()
        clf.train(X_train_feats, y_train,
            learning_rate=learningrate,reg=reg,num_iters=itertimes,batch_size= 1000,verbose=True)
        
        y_test_pred = clf.predict(X_val_feats)
        acc_val = np.mean(y_val == y_test_pred,dtype=np.float32)
        y_val_pred = clf.predict(X_train_feats)
        acc_train = np.mean(y_train == y_val_pred,dtype=np.float32)
        results[(learningrate,reg)] = acc_train,acc_val
        if acc_val > best_val:
            best_val = acc_val
            best_svm = clf
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)


iteration 0 / 5001: loss 8.999722
iteration 100 / 5001: loss 8.996549
iteration 200 / 5001: loss 8.991741
iteration 300 / 5001: loss 8.988790
iteration 400 / 5001: loss 8.986460
iteration 500 / 5001: loss 8.982367
iteration 600 / 5001: loss 8.978965
iteration 700 / 5001: loss 8.974984
iteration 800 / 5001: loss 8.971969
iteration 900 / 5001: loss 8.969142
iteration 1000 / 5001: loss 8.963446
iteration 1100 / 5001: loss 8.959726
iteration 1200 / 5001: loss 8.958336
iteration 1300 / 5001: loss 8.954537
iteration 1400 / 5001: loss 8.949623
iteration 1500 / 5001: loss 8.946146
iteration 1600 / 5001: loss 8.945355
iteration 1700 / 5001: loss 8.940049
iteration 1800 / 5001: loss 8.937382
iteration 1900 / 5001: loss 8.929915
iteration 2000 / 5001: loss 8.928838
iteration 2100 / 5001: loss 8.926349
iteration 2200 / 5001: loss 8.921660
iteration 2300 / 5001: loss 8.915756
iteration 2400 / 5001: loss 8.916440
iteration 2500 / 5001: loss 8.910866
iteration 2600 / 5001: loss 8.907423
iteration 270

iteration 2000 / 5001: loss 8.867997
iteration 2100 / 5001: loss 8.844170
iteration 2200 / 5001: loss 8.846077
iteration 2300 / 5001: loss 8.834147
iteration 2400 / 5001: loss 8.832737
iteration 2500 / 5001: loss 8.825976
iteration 2600 / 5001: loss 8.822345
iteration 2700 / 5001: loss 8.803999
iteration 2800 / 5001: loss 8.801998
iteration 2900 / 5001: loss 8.792176
iteration 3000 / 5001: loss 8.791411
iteration 3100 / 5001: loss 8.776366
iteration 3200 / 5001: loss 8.770175
iteration 3300 / 5001: loss 8.761716
iteration 3400 / 5001: loss 8.750084
iteration 3500 / 5001: loss 8.753218
iteration 3600 / 5001: loss 8.753151
iteration 3700 / 5001: loss 8.737773
iteration 3800 / 5001: loss 8.724840
iteration 3900 / 5001: loss 8.730905
iteration 4000 / 5001: loss 8.714374
iteration 4100 / 5001: loss 8.701032
iteration 4200 / 5001: loss 8.697807
iteration 4300 / 5001: loss 8.692774
iteration 4400 / 5001: loss 8.677631
iteration 4500 / 5001: loss 8.682417
iteration 4600 / 5001: loss 8.662387
i

iteration 3900 / 5001: loss 8.426616
iteration 4000 / 5001: loss 8.431711
iteration 4100 / 5001: loss 8.416813
iteration 4200 / 5001: loss 8.403207
iteration 4300 / 5001: loss 8.379688
iteration 4400 / 5001: loss 8.366380
iteration 4500 / 5001: loss 8.380426
iteration 4600 / 5001: loss 8.344755
iteration 4700 / 5001: loss 8.359921
iteration 4800 / 5001: loss 8.288800
iteration 4900 / 5001: loss 8.303645
iteration 5000 / 5001: loss 8.244236
lr 5.000000e-07 reg 5.000000e-04 train accuracy: 0.410265 val accuracy: 0.413000
lr 5.000000e-07 reg 1.000000e-03 train accuracy: 0.409429 val accuracy: 0.421000
lr 5.000000e-07 reg 2.000000e-03 train accuracy: 0.413469 val accuracy: 0.412000
lr 1.000000e-06 reg 5.000000e-04 train accuracy: 0.414245 val accuracy: 0.419000
lr 1.000000e-06 reg 1.000000e-03 train accuracy: 0.413388 val accuracy: 0.419000
lr 1.000000e-06 reg 2.000000e-03 train accuracy: 0.417592 val accuracy: 0.415000
lr 2.000000e-06 reg 5.000000e-04 train accuracy: 0.415122 val accuracy

In [8]:
# Evaluate your trained SVM on the test set
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

0.43


### Inline question 1:
Describe the misclassification results that you see. Do they make sense?


$\color{blue}{\textit Your Answer:}$The object contour of the image is similar or similar in color to the object contour of the misclassification class.




## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [9]:
# Preprocessing: Remove the bias dimension
# Make sure to run this cell only ONCE
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

(49000, 155)
(49000, 154)


In [10]:
from cs231n.classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 200
num_classes = 10
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
bestacc = 0
for hidden_size in [hidden_dim]:
    for learning_rate in [1e-1,5e-1]:
        for learning_rate_decay in [0.999]:
            for reg in [3e-4,1e-3,3e-3]:
                net = TwoLayerNet(input_dim, hidden_dim, num_classes)
                # Train the network
                stats = net.train(X_train_feats, y_train, X_val_feats, y_val,
                            num_iters=3000, batch_size=500,
                            learning_rate=learning_rate, learning_rate_decay=learning_rate_decay,
                            reg=reg, verbose=True)

                # Predict on the validation set
                val_acc = (net.predict(X_val_feats) == y_val).mean()
                print('hidden_size = %d,learning_rate = %f,learning_rate_decay = %f,reg = %f,Validation accuracy =%f '%(hidden_size,learning_rate,learning_rate_decay,reg,val_acc))
                if bestacc<val_acc:
                    bestacc = val_acc
                    best_net = net
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

iteration 0 / 3000: loss 2.302585
iteration 100 / 3000: loss 2.302109
iteration 200 / 3000: loss 2.172248
iteration 300 / 3000: loss 1.754953
iteration 400 / 3000: loss 1.636271
iteration 500 / 3000: loss 1.493881
iteration 600 / 3000: loss 1.422758
iteration 700 / 3000: loss 1.372279
iteration 800 / 3000: loss 1.377676
iteration 900 / 3000: loss 1.361315
iteration 1000 / 3000: loss 1.329902
iteration 1100 / 3000: loss 1.366421
iteration 1200 / 3000: loss 1.349177
iteration 1300 / 3000: loss 1.201482
iteration 1400 / 3000: loss 1.253556
iteration 1500 / 3000: loss 1.303853
iteration 1600 / 3000: loss 1.289811
iteration 1700 / 3000: loss 1.179719
iteration 1800 / 3000: loss 1.203061
iteration 1900 / 3000: loss 1.223234
iteration 2000 / 3000: loss 1.179431
iteration 2100 / 3000: loss 1.180226
iteration 2200 / 3000: loss 1.205678
iteration 2300 / 3000: loss 1.248767
iteration 2400 / 3000: loss 1.202628
iteration 2500 / 3000: loss 1.232928
iteration 2600 / 3000: loss 1.152092
iteration 270

In [11]:
# Run your best neural net classifier on the test set. You should be able
# to get more than 55% accuracy.

test_acc = (best_net.predict(X_test_feats) == y_test).mean()
print(test_acc)

0.568
